# Rešavanje SAT problema pomoću *minisat* rešavača

Za rešavanje SAT problemo koristimo *minisat* rešavač. Pri pokretanju *minisat* rešavača potrebno je navesti ulazni fajl u kome se nalazi 

Problem se zapisuje u *dimacs* zapisu i takav se prosleđuje *minisat* rešavaču koji nam vraća informaciju da li problem zadovoljin ili ne, i ako jeste zadovoljiv, vraća jedno rešenje.

**Dimacs zapis:**
*Dimacs* zapis se koristi za zapisivanje izrara koji su u komutativnoj normalnoj formi.
Pri red je oblika ```p cnf m n``` gde je ```m``` broj klauza u KNF-u, a ```n``` broj promenljivih koje se nalaze u KNF-u. Promenljive su označene brojevima od ```1``` do ```n```. U nastavku fajla se zapisuju sve klauze KNF-a, svaka u posebnom redu. Literali kaluze su odvojeni razmakom a na kraju svakog reda je potrebno zapisati nulu ```0```. Negirane promenljive se pišu sa minusom ispred oznake promenljive.

*Primer KNF-a zapisanog u dimacs zapisu:*
```
p cnf 2 3
-1 2
1 2 3
```

## Implementacija SAT rešavača u Pajtonu

In [6]:
import os

KNF predstavljamo klasom koja čuva svaku klauzu u obliku niske. Promeljive u klauzama ne moraju biti predstavljene brojevima kao što je to u *dimacs* zapisu, već klasa sama mapira imena promenjivih u odgovarajuće brojeve.

Ova klasa takođe sadrži pomoćnu funkciju koja vraća *dimacs* zapis tekućeg KNF-a.

In [34]:
class CNF:
    
    def __init__(self):
        self.clauses = []
        self.var_to_num = {}
        self.num_to_var = {}
        
    def add_clause(self, clause):
        
        for literal in clause:
            var = literal.strip("-")
            if var not in self.var_to_num:
                var_number = len(self.num_to_var) + 1
                self.var_to_num[var] = var_number
                self.num_to_var[var_number] = var
                
        self.clauses.append(clause)
    
    def dimacs(self):
        
        cnf = f"p cnf {len(self.clauses)} {len(self.var_to_num)}\n"
        
        for clause in self.clauses:
            for literal in clause:
                
                var = literal.strip('-')
                
                if literal.startswith('-'):
                    cnf += '-'
                cnf += f"{str(self.var_to_num[var])} "
            
            cnf += "0\n"
            
        return cnf
    
    def get_var_name(self, var_num):
        return self.num_to_var[var_num]
    
    def get_var_num(self, var_name):
        return self.var_to_num[var_name]
    

In [41]:
def minisat_solve(problem_name, cnf):
    
    with open(f"{problem_name}.cnf", "w") as cnf_input_file:
        cnf_input_file.write(cnf.dimacs())
        
    os.system(f"minisat {problem_name}.cnf {problem_name}_result.cnf")
    
    with open(f"{problem_name}_result.cnf", "r") as cnf_output_file:
        cnf_output_lines = cnf_output_file.readlines()
    
    
    result = ""
    if cnf_output_lines[0].startswith("SAT"):
        
        result += "SAT\n"
        literals = cnf_output_lines[1].split(" ")
        
        for literal in literals:
            if not literal.startswith('0'):
                var_num = literal.strip('-')

                if literal.startswith('-'):
                    result += '-'
                result += cnf.get_var_name(int(var_num)) + " "
                
    elif cnf_output_lines[0].startswith("UNSAT"):
        result += "UNSAT\n"
        
    return result

## Primer upotrebe

Zapisujemo KNF: $ (\neg a \lor b) \land (\neg b \lor c) $

In [42]:
cnf = CNF()
cnf.add_clause(["-a", "b"])
cnf.add_clause(["-b", "c"])

print(minisat_solve("sample", cnf))

============================[ Problem Statistics ]=============================
|                                                                             |
|  Number of variables:             3                                         |
|  Number of clauses:               2                                         |
|  Parse time:                   0.00 s                                       |
|  Eliminated vars:                 3                                           |
|  Vars set       :                 0                                           |
|  Eliminated clauses:           0.00 Mb                                      |
|  Simplification time:          0.00 s                                       |
|                                                                             |
============================[ Search Statistics ]==============================
| Conflicts |          ORIGINAL         |          LEARNT          | Progress |
|           |    Vars  Clauses Liter

Interpretacija: ```a = 0, b = 0, c = 1```